<a href="https://colab.research.google.com/github/RyuMyunggi/NASA-battery-dataset-eda/blob/main/nasa_data_set_save_to_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# import 
import datetime
import json
import numpy as np
from scipy.io import loadmat
from os import path
from google.colab import drive

In [4]:
# eda for nasa data set

def eda_for_nasa_data_set(battery):
  mat_data = loadmat(f'/content/drive/MyDrive/NASA-data-set/{battery}.mat')
  nasa_data = mat_data[battery][0][0][0][0]

  print('data type: ', type(nasa_data))
  # data type:  <class 'numpy.ndarray'>
  print('d type: ', nasa_data.dtype)
  # d type: [('type', 'O'), ('ambient_temperature', 'O'), ('time', 'O'), ('data', 'O')]
  print('shape:', nasa_data.shape)
  # shape: (616,)
  print('ndim:', nasa_data.ndim)
  # ndim: 1
  print('data size: ', nasa_data.size)
  # data size:  616
  print('memory size', nasa_data.nbytes)
  # memory size 19712

# run
eda_for_nasa_data_set('B0005')

data type:  <class 'numpy.ndarray'>
d type:  [('type', 'O'), ('ambient_temperature', 'O'), ('time', 'O'), ('data', 'O')]
shape: (616,)
ndim: 1
data size:  616
memory size 19712


In [7]:
# preprocessing and save data

def get_nasa_data_set(battery):
    # return
    res = {
        'charge': {},
        'discharge': {},
        'impedance': {}
    }

    mat_data = loadmat(f'/content/drive/MyDrive/NASA-data-set/{battery}.mat')
    mess = mat_data[battery][0][0][0][0]
    for i, element in enumerate(mess):
        # battery status 
        # (charge / discharge / impedance)
        condition = element[0][0] 
        data = element[3]

        if condition == 'charge':
            res['charge'][str(i)] = {}
            res['charge'][str(i)]['amb_temp'] = str(element[1][0][0])
            res['charge'][str(i)]['date_time'] = get_datetime(element)
            res['charge'][str(i)]['voltage_battery'] = data[0][0][0][0].tolist()
            res['charge'][str(i)]['current_battery'] = data[0][0][1][0].tolist()
            res['charge'][str(i)]['temp_battery'] = data[0][0][2][0].tolist()
            res['charge'][str(i)]['current_load'] = data[0][0][3][0].tolist()
            res['charge'][str(i)]['voltage_load'] = data[0][0][4][0].tolist()
            res['charge'][str(i)]['time'] = data[0][0][5][0].tolist()

        elif condition == 'discharge':
            res['discharge'][str(i)] = {}
            res['discharge'][str(i)]['amb_temp'] = str(element[1][0][0])
            res['discharge'][str(i)]['date_time'] = get_datetime(element)
            res['discharge'][str(i)]['voltage_battery'] = data[0][0][0][0].tolist()
            res['discharge'][str(i)]['current_battery'] = data[0][0][1][0].tolist()
            res['discharge'][str(i)]['temp_battery'] = data[0][0][2][0].tolist()
            res['discharge'][str(i)]['current_load'] = data[0][0][3][0].tolist()
            res['discharge'][str(i)]['voltage_load'] = data[0][0][4][0].tolist()
            res['discharge'][str(i)]['time'] = data[0][0][5][0].tolist()

        else: # impedance
            res['impedance'][str(i)] = {}
            res['impedance'][str(i)]["amb_temp"] = str(element[1][0][0])
            res['impedance'][str(i)]['date_time'] = get_datetime(element)
            res['impedance'][str(i)]["amb_temp"] = str(element[1][0][0])
            res['impedance'][str(i)]["sense_current"] = {}
            res['impedance'][str(i)]["battery_current"] = {}
            res['impedance'][str(i)]["current_ratio"] = {}
            res['impedance'][str(i)]["battery_impedance"] = {}
            res['impedance'][str(i)]["rectified_impedance"] = {}
            res['impedance'][str(i)]["sense_current"]["real"] = np.real(data[0][0][0][0]).tolist()
            res['impedance'][str(i)]["sense_current"]["imag"] = np.imag(data[0][0][0][0]).tolist()
            res['impedance'][str(i)]["battery_current"]["real"] = np.real(data[0][0][1][0]).tolist()
            res['impedance'][str(i)]["battery_current"]["imag"] = np.imag(data[0][0][1][0]).tolist()
            res['impedance'][str(i)]["current_ratio"]["real"] = np.real(data[0][0][2][0]).tolist()
            res['impedance'][str(i)]["current_ratio"]["imag"] = np.imag(data[0][0][2][0]).tolist()
            res['impedance'][str(i)]["battery_impedance"]["real"] = np.real(data[0][0][3]).tolist()
            res['impedance'][str(i)]["battery_impedance"]["imag"] = np.imag(data[0][0][3]).tolist()
            res['impedance'][str(i)]["rectified_impedance"]["real"] = np.real(data[0][0][4]).tolist()
            res['impedance'][str(i)]["rectified_impedance"]["imag"] = np.imag(data[0][0][4]).tolist()
            res['impedance'][str(i)]["re"] = float(data[0][0][5][0][0])
            res['impedance'][str(i)]["rct"] = float(data[0][0][6][0][0])

    return res

# datetime_format
def get_datetime(element):
    year = int(element[2][0][0])
    month = int(element[2][0][1])
    day = int(element[2][0][2])
    hour = int(element[2][0][3])
    minute = int(element[2][0][4])
    second = int(element[2][0][5])
    millisecond = int((second % 1) * 1000)
    date_time = datetime.datetime(year, month, day, hour, minute, second, millisecond)

    return date_time.strftime("%d %b %Y %H:%M:%S")

# save data as json
def save_json(data, title):
  drive.mount('/content/drive')
  file_path = path.join('./drive/MyDrive/NASA-processing-data', title)
  with open(file_path + '.json', 'w') as f:
      json.dump(data, f, indent=4)

# run
# NASA Data Set 1
for battery in ['B0005', 'B0006', 'B0007', 'B0018']:
  res = get_nasa_data_set(battery)
  for key in res:
    data = res[key]
    save_json(data, f'{battery}_{key}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/conte